In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from data_manager import *
from metadata import *
from distributed_utilities import *

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD642/MD642_anchor.txt
File does not exist: 

In [2]:
classifier_id = 30

In [3]:
stack = 'MD591'
first_sec, last_sec = metadata_cache['section_limits'][stack]

In [ ]:
wait_num_nodes(16)

In [7]:
t = time.time()
sys.stderr.write('running svm classifier ...')

run_distributed(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'apply_classifiers_v3.py'),
                'stack': stack,
                'setting': classifier_id},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                argument_type='partition',
               cluster_size=16,
               jobs_per_node=1)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 302s

running svm classifier ...

Setting autoscaling group cfncluster-yuncongCluster-ComputeFleet-1VBMCN42UD1WL capaticy to 16...it may take more than 5 minutes for SGE to know new hosts.


16 nodes requested, 15 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 302.817464 seconds


In [33]:
t = time.time()
sys.stderr.write('Resampling scoremaps ...')

downscale = 8

run_distributed(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(setting)d %(downscale)d' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'resample_scoremaps_v3.py'),
                'stack': stack,
                'setting': classifier_id,
                'downscale': downscale},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                argument_type='partition',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) #

Resampling scoremaps ...

Setting autoscaling group cfncluster-yuncongCluster-ComputeFleet-1VBMCN42UD1WL capaticy to 16...it may take more than 5 minutes for SGE to know new hosts.


16 nodes requested, 15 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 247.409529 seconds


In [ ]:
t = time.time()
sys.stderr.write('visualize scoremaps ...')

add_label_text = False
viz_downscale = 8

run_distributed(command='%(script_path)s %(stack)s %(setting)d -b %%(first_sec)d -e %%(last_sec)d -d %(downscale)d %(add_label_text)s' % \
                {'script_path': os.path.join(REPO_DIR, 'learning', 'visualize_scoremaps_v3.py'),
                'stack': stack,
                 'setting': classifier_id,
                'add_label_text': '-a' if add_label_text else '',
                'downscale': viz_downscale},
                kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                argument_type='partition',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

In [46]:
t = time.time()
sys.stderr.write('constructing score volumes ...')

volume_downscale = 32

run_distributed(command='%(script_path)s %(stack)s %%(structure)s %(setting)d %(downscale)d' % \
                {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'construct_score_volume_v4.py'),
                'stack': stack,
                'setting': classifier_id,
                'downscale': volume_downscale},
                kwargs_list=dict(structure=all_known_structures),
                argument_type='single',
                cluster_size=16)

wait_qsub_complete()

sys.stderr.write('done in %f seconds\n' % (time.time() - t))# 36s

constructing score volumes ...

Setting autoscaling group cfncluster-yuncongCluster-ComputeFleet-1VBMCN42UD1WL capaticy to 16...it may take more than 5 minutes for SGE to know new hosts.


16 nodes requested, 15 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 36.286749 seconds


In [59]:
t0 = time.time()
sys.stderr.write('compute score volume gradients...')

stack = 'MD591'
downscale = 32

for structure in all_known_structures:
    print structure
    try:
        volume = DataManager.load_score_volume(stack=stack, structure=structure, downscale=downscale,
                                               classifier_setting=classifier_id)

        t = time.time()

        gy_gx_gz = np.gradient(volume.astype(np.float32), 3, 3, 3)
        # 3.3 second - re-computing is much faster than loading
        # .astype(np.float32) is important;
        # Otherwise the score volume is type np.float16, np.gradient requires np.float32 and will have to convert which is very slow
        # 2s (float32) vs. 20s (float16)

        sys.stderr.write('Gradient %s: %f seconds\n' % (structure, time.time() - t))

        t = time.time()

        gx_fp = DataManager.get_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, classifier_setting=classifier_id, suffix='gx')
        gy_fp = DataManager.get_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, classifier_setting=classifier_id, suffix='gy')
        gz_fp = DataManager.get_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, classifier_setting=classifier_id, suffix='gz')

        create_parent_dir_if_not_exists(gx_fp)

        bp.pack_ndarray_file(gy_gx_gz[1], gx_fp)
        bp.pack_ndarray_file(gy_gx_gz[0], gy_fp)
        bp.pack_ndarray_file(gy_gx_gz[2], gz_fp)

        upload_from_ec2_to_s3(gx_fp)
        upload_from_ec2_to_s3(gy_fp)
        upload_from_ec2_to_s3(gz_fp)

        del gy_gx_gz

        sys.stderr.write('save %s: %f seconds\n' % (structure, time.time() - t))

    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error computing gradient for %s.\n' % structure)
        
sys.stderr.write('done in %f seconds\n' % (time.time() - t0)) # 70 seconds

compute score volume gradients...

5N


Gradient 5N: 1.495417 seconds


aws s3 cp /shared/CSHL_volumes/MD591/MD591_down32_scoreVolume_clf_30/score_volume_gradients/MD591_down32_scoreVolume_clf_30_5N_gx.bp s3://mousebrainatlas-data/CSHL_volumes/MD591/MD591_down32_scoreVolume_clf_30/score_volume_gradients/MD591_down32_scoreVolume_clf_30_5N_gx.bp


Child returned 0
1.53 seconds.


...

Child returned 0
1.44 seconds.


aws s3 cp /shared/CSHL_volumes/MD591/MD591_down32_scoreVolume_clf_30/score_volume_gradients/MD591_down32_scoreVolume_clf_30_IC_gy.bp s3://mousebrainatlas-data/CSHL_volumes/MD591/MD591_down32_scoreVolume_clf_30/score_volume_gradients/MD591_down32_scoreVolume_clf_30_IC_gy.bp


Child returned 0
1.40 seconds.


aws s3 cp /shared/CSHL_volumes/MD591/MD591_down32_scoreVolume_clf_30/score_volume_gradients/MD591_down32_scoreVolume_clf_30_IC_gz.bp s3://mousebrainatlas-data/CSHL_volumes/MD591/MD591_down32_scoreVolume_clf_30/score_volume_gradients/MD591_down32_scoreVolume_clf_30_IC_gz.bp


Child returned 0
1.41 seconds.
save IC: 8.828161 seconds
done in 310.705719 seconds


In [56]:
# t = time.time()
# sys.stderr.write('compute score volume gradients...')

# run_distributed(command='%(script_path)s %(stack)s %%(structure)s %(setting)d %(downscale)d' % \
#                 {'script_path': os.path.join(REPO_DIR, 'reconstruct', 'compute_score_volume_gradients_v4.py'),
#                 'stack': stack,
#                 'setting': classifier_id,
#                 'downscale': volume_downscale},
#                 kwargs_list=dict(structure=all_known_structures),
#                 argument_type='single',
#                 cluster_size=16)

# wait_qsub_complete()

# sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 70 seconds

compute score volume gradients...

Setting autoscaling group cfncluster-yuncongCluster-ComputeFleet-1VBMCN42UD1WL capaticy to 16...it may take more than 5 minutes for SGE to know new hosts.


16 nodes requested, 15 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
done in 156.988350 seconds


In [55]:
! tail /home/ubuntu/stderr_2.log

File does not exist: /shared/CSHL_data_processed/MD642/MD642_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD642/MD642_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD642/MD642_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD642/MD642_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_cropbox.txt
Gradient LC: 1.565091 seconds
save LC: 46.780748 seconds
